In [1]:
%load_ext autoreload
%autoreload 2

%load_ext lab_black

In [2]:
import sys
from pathlib import Path

In [3]:
_pwd = Path(".").resolve()
print(_pwd)
sys.path.append(str(_pwd))

/data/ik18445_cache/projects/phenotype-mapping/analysis/notebooks/mvp_ontology_distance


In [4]:
import json

import pandas as pd
import janitor
from pydash import py_

In [5]:
from common_funcs import utils

In [6]:
proj_root = utils.find_project_root("docker-compose.yml")
assert proj_root.exists(), proj_root

data_dir = proj_root / "data"
artifacts_dir = proj_root / "data" / "output" / "mvp-efo-terms-2022-12"
assert artifacts_dir.exists(), artifacts_dir

In [7]:
efo_terms = pd.read_csv(artifacts_dir / "efo_terms.csv")
efo_terms

,efo_id,efo_term,idx,efo_term_clean
0,CHEBI_16737,creatinine,0,creatinine
1,CHEBI_16990,bilirubin,1,bilirubin
2,CHEBI_17347,testosterone,2,testosterone
3,CHEBI_23965,estradiol,3,estradiol
4,CHEBI_26020,phosphate,4,phosphate
...,...,...,...,...
11086,UBERON_0000947,aorta,11086,aorta
11087,UBERON_0001467,shoulder,11087,shoulder
11088,UBERON_0002198,neurohypophysis,11088,neurohypophysis
11089,UBERON_0002384,connective tissue,11089,connective tissue


In [8]:
distance = pd.read_csv(artifacts_dir / "distance.csv")
distance

,id_a,id_b,cosine_similarity
0,CHEBI_16737,CHEBI_16990,0.7246
1,CHEBI_16737,CHEBI_17347,0.3148
2,CHEBI_16737,CHEBI_23965,0.1639
3,CHEBI_16737,CHEBI_26020,0.2283
4,CHEBI_16737,CHEBI_35475,0.0281
...,...,...,...
61499590,UBERON_0001467,UBERON_0002384,0.1321
61499591,UBERON_0001467,UBERON_0010996,0.6020
61499592,UBERON_0002198,UBERON_0002384,0.1815
61499593,UBERON_0002198,UBERON_0010996,0.0923


In [9]:
TERM = "body mass index"
# TERM = "alcohol drinking"
# TERM = "cancer"

bmi_ent = efo_terms[efo_terms["efo_term_clean"] == TERM].reset_index(drop=True)
bmi_ent

,efo_id,efo_term,idx,efo_term_clean
0,EFO_0004340,body mass index,601,body mass index


In [10]:
ent_id = bmi_ent["efo_id"][0]


subset = distance[
    (distance["id_a"] == ent_id) | (distance["id_b"] == ent_id)
].sort_values("cosine_similarity", ascending=False)
print(len(subset))

subset1 = subset[:30]


top_df = pd.DataFrame(
    [
        {
            "id": _["id_a"] if _["id_a"] != ent_id else _["id_b"],
            "score": _["cosine_similarity"],
        }
        for _ in subset1.to_dict(orient="records")
    ]
).merge(efo_terms[["efo_id", "efo_term"]].rename(columns={"efo_id": "id"}), on="id")
top_df

11090


,id,score,efo_term
0,EFO_0005851,0.9553,height-adjusted body mass index
1,EFO_0007041,0.9129,obese body mass index status
2,EFO_0005935,0.9039,overweight body mass index status
3,EFO_0004995,0.9011,lean body mass
4,EFO_0005409,0.8979,fat body mass
5,EFO_0005936,0.8942,underweight body mass index status
6,EFO_0009890,0.8583,lean mass-adjusted fat body mass
7,EFO_0009805,0.8367,whole body water mass
8,EFO_0004339,0.8215,body height
9,EFO_0004338,0.8182,body weight


In [13]:
with (artifacts_dir / "encode.json").open() as f:
    encode = json.load(f)

print(len(encode[0]["term_vector"]))

200
